In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
import pandas as pd
import re

df = pd.read_csv("data/test.csv")

In [ ]:
def suppr_espace(text):
        # \s = any whitespace | + = one or more
        regex = re.compile(r"\s+")
        text = re.sub(regex, "", str(text))
        return text


def recup_nom_auteur(text):
    try:
        regex = re.compile(r",")
        index_auteur = re.search(regex, text)
        return text[:index_auteur.end()-1]

    except AttributeError:
        return text


def efface_entete(text):
    try:
        regex = re.compile(r"Le taux de reconnaissance estimé pour ce document est de [0-9]{2,}%.")
        fin_entete = re.search(regex, str(text)).end()

    except AttributeError:
        return text
    
    regex = re.compile(r"En savoir plus sur l\'OCR")
    text = re.sub(regex, "", text)
    return text[fin_entete:]


def retire_saut_ligne(text):
    regex = re.compile(r"\n")
    text = re.sub(regex, " ", str(text))
    return text

In [ ]:
# Application des fonctions
df["texte"] = df["texte"].apply(efface_entete).apply(retire_saut_ligne)
df["auteur"] = df["auteur"].apply(recup_nom_auteur).apply(suppr_espace)
df["sujet"] = df["sujet"].apply(suppr_espace)

In [6]:
# Analyse TF-IDF
vectorizer = TfidfVectorizer(max_df=.65, min_df=1, stop_words=None, use_idf=True, norm=None)
transformed_documents = vectorizer.fit_transform(df["texte"].tolist())

transformed_documents_as_array = transformed_documents.toarray()

transformed_documents_as_array[0]

for counter, doc in enumerate(transformed_documents_as_array):
    # construct a dataframe
    tf_idf_tuples = list(zip(vectorizer.get_feature_names_out(), doc))
    one_doc_as_df = pd.DataFrame.from_records(tf_idf_tuples, columns=['term', 'score']).sort_values(by='score', ascending=False).reset_index(drop=True)
    
    break

In [7]:
# Export
with open("corpus4.txt", "w") as c:
    for texte, auteur, sujet in zip(df["texte"], df["auteur"], df["sujet"]):
        c.write(f"**** *auteur_{auteur} *sujet_{sujet}\n {texte}\n\n")

with open("corpus.txt", "r") as c:
    file = c.read()
    print(len(file))

2617401
